In [1]:
import librosa 
import numpy as np

def extract_features(file_path, max_pad_len=174):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        
        # Pad or truncate
        if mfccs.shape[1] < max_pad_len:
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_pad_len]
            
        return mfccs
    except Exception as e:
        print(f"❌ Error: {e} | File: {file_path}")
        return None


In [2]:
import glob
import csv

def load_dataset_from_dir(base_dir):
    X, y = [], []
    for label in os.listdir(base_dir):
        label_dir = os.path.join(base_dir, label)
        if not os.path.isdir(label_dir): 
            continue
        for path in glob.glob(os.path.join(label_dir, "*.wav")):
            try:
                y_audio, sr = load_audio(path)
                feats = extract_features(y_audio, sr)
                X.append(feats)
                y.append(label)
            except Exception as e:
                print("Failed:", path, e)
    return np.array(X), np.array(y)

In [3]:
import glob, os

data = []
labels = []

emotion_map = {
    "01": "neutral", "02": "calm", "03": "happy", "04": "sad",
    "05": "angry", "06": "fearful", "07": "disgust", "08": "surprised"
}

files = glob.glob("../data/raw/kaggle_speech/**/*.wav", recursive=True)
files = [os.path.normpath(f) for f in files]  # normalize slashes

# import librosa

# test_file = files[0]
# print("Testing file:", test_file)

# y, sr = librosa.load(test_file, sr=None)  # sr=None preserves original sampling rate
# print("Loaded audio:", y.shape, "Sampling rate:", sr)

for file in files:
    file = os.path.normpath(file)
    file_name = os.path.splitext(os.path.basename(file))[0]  # remove .wav
    emotion = file_name.split("-")[2]  # get emotion code
    label = emotion_map[emotion]

    feature = extract_features(file)
    if feature is not None:
        data.append(feature)
        labels.append(label)



In [36]:
pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [20]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X = np.array(data)
y = np.array(labels)

lb = LabelEncoder()
y = to_categorical(lb.fit_transform(y))


ModuleNotFoundError: No module named 'tensorflow'

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# CNN expects 4D input
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

ModuleNotFoundError: No module named 'speech_tone_pipeline'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(40, 174, 1)),
    MaxPooling2D((2,2)),
    Dropout(0.3),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_test, y_test))


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)
